# Demo Notebook for deploying CLIPTextModel to OpenSearch

#### [Download notebook](placeholder_for_url)

Related Docs:
* [OpenSearch ML Framework](https://opensearch.org/docs/latest/ml-commons-plugin/ml-framework/)
* [Huggingface - CLIP](https://huggingface.co/docs/transformers/model_doc/clip)
* [Huggingface - export to torchscript](https://huggingface.co/docs/transformers/torchscript)

This notebook provides a walkthrough for users to trace, register, and deploy a CLIPTextModel from a local file. CLIPTextModel can be used with the [Neural Search](https://opensearch.org/docs/latest/search-plugins/neural-search/) plugin to generate embeddings of documents and ingest time and of user queries at search time. 

Step 0: Import packages and set up client

Step 1: Trace CLIPTextModel and export to TorchScript

Step 2: Prep files for registration

Step 3: Register model to OpenSearch

Step 4: Deploy model

## Step 0: Import packages and set up client

In [52]:
from transformers import CLIPProcessor, CLIPTextModel
import torch

import opensearch_py_ml as oml
from opensearch_py_ml.ml_commons import MLCommonClient
from opensearchpy import OpenSearch

import warnings
warnings.filterwarnings("ignore", message="Unverified HTTPS request")

In [53]:
# Connect to OpenSearch cluster
host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.

def get_os_client(cluster_url = CLUSTER_URL,username='admin',password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts = [{'host': host, 'port': port}],
        http_compress = True, # enables gzip compression for request bodies
        http_auth = auth,
        use_ssl = False,
        verify_certs = False,
        ssl_assert_hostname = False,
        ssl_show_warn = False,
    )
    return client

In [54]:
client = get_os_client()
ml_client = MLCommonClient(client)

## Step 1: Trace CLIPTextModel and export to TorchScript

To use a model in OpenSearch, you’ll need to export the model into a portable format. As of Version 2.5, OpenSearch only supports the TorchScript and ONNX formats.

Exporting a model to TorchScript requires two things:

* model instantiation with the torchscript flag
* a forward pass with dummy inputs

The dummy inputs are used for a model's forward pass. While the inputs’ values are propagated through the layers, PyTorch keeps track of the different operations executed on each tensor. These recorded operations are then used to create the trace of the model.

As of OpenSearch 2.6, the ML Framework supports text-embedding models only. CLIP is multi-modal, but we will use CLIPTextModel only here.

In [55]:
model_name = "openai/clip-vit-base-patch32" #See https://huggingface.co/models for other options 
text_to_encode = "example search query" #See https://huggingface.co/docs/transformers/torchscript for more info on dummy inputs

# Instantiate CLIPTextModel and CLIPProcessor with pretrained weights
model = CLIPTextModel.from_pretrained(model_name, torchscript=True, return_dict=False)
processor = CLIPProcessor.from_pretrained(model_name)

# Use processor to generate tensors and create dummy input
text_inputs =processor(text=text_to_encode, return_tensors="pt",max_length=77, padding="max_length", truncation=True)
dummy_input = [text_inputs['input_ids'], text_inputs['attention_mask']]

# Trace model and convert to torchscript object
traced_model = torch.jit.trace(model, dummy_input)

# Save model in portable format
torch.jit.save(traced_model, "traced_model_example.pt")

c:\Users\patri\anaconda3\envs\opensearch_ml\lib\site-packages\transformers\models\clip\modeling_clip.py:287: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
c:\Users\patri\anaconda3\envs\opensearch_ml\lib\site-packages\transformers\models\clip\modeling_clip.py:295: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_attention_mask.size() != (bsz, 1, tgt_len, src_len):
c:\Users\patri\anaconda3\envs\opensearch_ml\lib\site-packages\transformers\models\clip\modeling_clip.py:304: TracerWa

## Step 2: Prep files for registration

OpenSearch requires two files zipped together for registration:
* Model in TorchScript format
* tokenizor.json file

The tokenizor for the model used in this example can be found [here](https://huggingface.co/openai/clip-vit-base-patch32/tree/main)

Additionally, a config.json file with the following details must be passed with the .zip. More info on [model config](clip-vit-base-patch32) 

In [57]:
# config.json sample contents
"""
{
    "name": "clip-vit-base-patch32",
    "version": '1.0.0',
    "model_format": "TORCH_SCRIPT",
    "model_config": {
        "model_type": "clip",
        "embedding_dimension": 512,
        "framework_type": "huggingface_transformers"
    }
}
"""

'\n{\n    "name": "clip-vit-base-patch32",\n    "version": \'1.0.0\',\n    "model_format": "TORCH_SCRIPT",\n    "model_config": {\n        "model_type": "clip",\n        "embedding_dimension": 512,\n        "framework_type": "huggingface_transformers"\n    }\n}\n'

## Step 3: Register model to OpenSearch

* Model name in config.json should match .pt torchscript file name
* Record the model ID from the output of the next cell


In [1]:
model_path = "<your_path>/<your_zipped_folder>.zip"
model_config_path = "<your_path>/config.json"

model_id_file_system = ml_client.register_model(model_path, model_config_path, isVerbose=True, deploy_model = False)


NameError: name 'ml_client' is not defined

: 

## Step 4: Deploy model


In [60]:
model_id = 'JHtkIBlaN3lvMRH8H' #your model ID from previous step
ml_client.deploy_model(model_id)

task_id: JXKtkIoBlaNlvM5RgN8J
Model deployed successfully


{'model_id': 'JHKtkIoBlaNlvM5RHN8H',
 'task_type': 'DEPLOY_MODEL',
 'function_name': 'TEXT_EMBEDDING',
 'state': 'COMPLETED',
 'worker_node': ['4K6CeIPPTkKiwZMplvJ6CQ'],
 'create_time': 1694644404233,
 'last_update_time': 1694644410593,
 'is_async': True}

In [61]:
# Check model status
ml_client.get_model_info(model_id)

{'name': 'traced_model_example2',
 'model_group_id': 'I3KtkIoBlaNlvM5RG9_f',
 'algorithm': 'TEXT_EMBEDDING',
 'model_version': '1',
 'model_format': 'TORCH_SCRIPT',
 'model_state': 'DEPLOYED',
 'model_content_size_in_bytes': 186971303,
 'model_content_hash_value': 'd478498a68f19acbee873b07b533a43b5efbfc75af70a9e0c3a7564aa09ab389',
 'model_config': {'model_type': 'clip',
  'embedding_dimension': 512,
  'framework_type': 'HUGGINGFACE_TRANSFORMERS'},
 'created_time': 1694644378631,
 'last_updated_time': 1694644410593,
 'last_deployed_time': 1694644410593,
 'total_chunks': 19,
 'planning_worker_node_count': 1,
 'current_worker_node_count': 1,
 'planning_worker_nodes': ['4K6CeIPPTkKiwZMplvJ6CQ'],
 'deploy_to_all_nodes': True}